In [31]:
import pydot


def walk_dictionaryv2(graph, dictionary, parent_node=None):
    '''
    Recursive plotting function for the decision tree stored as a dictionary
    '''

    for k in dictionary.keys():

        if parent_node is not None:

            from_name = parent_node.get_name().replace("\"", "") + '_' + str(k)
            from_label = str(k)

            node_from = pydot.Node(from_name, label=from_label)
            graph.add_node(node_from)
            graph.add_edge( pydot.Edge(parent_node, node_from) )

            if isinstance(dictionary[k], dict): # if interim node


                walk_dictionaryv2(graph, dictionary[k], node_from)

            else: # if leaf node
                to_name = str(k) + '_' + str(dictionary[k]) # unique name
                to_label = str(dictionary[k])

                node_to = pydot.Node(to_name, label=to_label, shape='box', style = 'filled', fillcolor = '#CCCDC6' if isinstance(dictionary[k], dict) else '#CCCDC6')
                graph.add_node(node_to)
                graph.add_edge(pydot.Edge(node_from, node_to))

                #node_from.set_name(to_name)

        else:

            from_name =  str(k)
            from_label = str(k)

            node_from = pydot.Node(from_name, label=from_label)
            walk_dictionaryv2(graph, dictionary[k], node_from)


def plot_tree(tree, name):

    # first you create a new graph, you do that with pydot.Dot()
    graph = pydot.Dot(graph_type='graph')

    walk_dictionaryv2(graph, tree)

    graph.write_png(name+'.png')



In [32]:
#https://stackoverflow.com/questions/24657384/plotting-a-decision-tree-with-pydot
import math
import numpy as np
import pandas as pd

class Node:
	'''
	Data structure is to store nodes of tree.
	'''
	def __init__(self, is_leaf=False, criterion=None, label="", threshold=None, pure_degree="", correct_wrong=None, children=[]):
		self.criterion = criterion
		self.label = label
		self.threshold = threshold
		self.is_leaf = is_leaf
		self.children = children
		self.pure_degree = pure_degree
		self.correct_wrong = correct_wrong
class C45:
	"""Creates a bi-decision tree with C4.5 algorithm"""

	def __init__(self):
		self.data = None
		self.classes = None
		self.numAttributes = None
		self.attributes = None
		self.tree = None
		self.tree_dict={}

	def load_data(self, attributes, data, classes):
		self.data = data
		self.classes = classes
		self.numAttributes = len(attributes)
		self.attributes = attributes

	def printTree(self):
		'''
		Visualize on console
		'''
		self.__printNode(self.tree)
	
	def __printNode(self, node, indent=""):
		if node.is_leaf: 
			return 
		
		leftChild = node.children[0]
		rightChild = node.children[1]
		if leftChild.is_leaf:
			print(indent+"|____" + node.criterion + " <= " + str(node.threshold) + " : " + leftChild.label + " (" +str(leftChild.pure_degree)+ ")")
		else:
			print(indent+"|____" + node.criterion + " <= " + str(node.threshold))
		self.__printNode(leftChild, indent + "|	")
		
		if rightChild.is_leaf:
			print(indent+"|____" + node.criterion + " > " + str(node.threshold) + " : " + rightChild.label+ " (" + str(rightChild.pure_degree) + ")")
			print(indent)
		else:
			print(indent+"|____" + node.criterion + " > " + str(node.threshold))
		self.__printNode(rightChild, indent + "	")
	
	def __generate_tree_dict(self):
		'''
		Generate dictionary for visualizing in image
		'''
		self.tree_dict[self.tree.criterion] = self.__recursive_generate_tree_dict(self.tree)
	
	def __recursive_generate_tree_dict(self, node:Node):
		if node.is_leaf:
			return 

		leftChild:Node = node.children[0]
		rightChild:Node = node.children[1]

		branch ={}
		if leftChild.is_leaf:
			branch["<="+str(node.threshold)] = leftChild.label + "\n"+str(round(leftChild.pure_degree,2)) + "\n" +str(leftChild.correct_wrong)
		else:
			branch["<="+str(node.threshold)] = {leftChild.criterion: self.__recursive_generate_tree_dict(leftChild)}
		
		if rightChild.is_leaf:
			branch[">"+str(node.threshold)] = rightChild.label+ "\n"+str(round(rightChild.pure_degree,2))+ "\n" +str(rightChild.correct_wrong)
		else:
			branch[">"+str(node.threshold)] = {rightChild.criterion: self.__recursive_generate_tree_dict(rightChild)}

		return branch
	
	def draw_tree(self, name):
		'''
		input: 
			name: name of image of tree 
		visualize tree with image named "name"
		'''
		self.__generate_tree_dict()
		plot_tree(self.tree_dict, name)

	def generate_tree(self, is_prune=False):
		'''
		run generating tree from data.
		Input:
			is_prune: prune the leaf if two branches are the same label
		'''
		self.tree = self.__recursiveGenerateTree(curData=self.data, curAttributes=self.attributes,  is_prune=is_prune)

	def __recursiveGenerateTree(self, curData=None, curAttributes=None, criterion=None, threshold=None, is_prune=False):
		if len(curData) == 0:
			#No any data sample for this curAttributes. (only in decrete criterion)
			return None

		is_pure, class_ = self.__allSameClass(curData)
		if is_pure:
			return Node(is_leaf=True, criterion=criterion, label=class_, pure_degree=100.0, correct_wrong=(len(curData), len(curData)-len(curData)))
		elif len(curAttributes) == 0:
			main_class, pure_degree, true_class_num = self.get_main_class(curData)
			return Node(is_leaf=True, criterion=criterion, label=main_class, pure_degree=pure_degree, correct_wrong=(true_class_num, len(curData) - true_class_num))
		else:
			(best_attr, threshold_, Sis) = self.__split_data(curData, curAttributes)
			remainingAttributes = curAttributes[:]
			remainingAttributes.remove(best_attr)
			children = [self.__recursiveGenerateTree(Si, remainingAttributes, criterion=best_attr, threshold=threshold_, is_prune=is_prune) for Si in Sis]
			if len(children)==1:
				node=children[0]
				node.criterion=criterion
				node.threshold = threshold
				
			elif (is_prune and children[0].is_leaf==True 
				and children[1].is_leaf==True 
				and children[0].label==children[1].label):

				correct_wrong = (children[0].correct_wrong[0] + children[1].correct_wrong[0], children[0].correct_wrong[1]+children[1].correct_wrong[1])
				node = Node(is_leaf=True, criterion=None,
							label=children[0].label,
				            children=[], pure_degree=correct_wrong[0]/(correct_wrong[0]+correct_wrong[1]),
                            correct_wrong=correct_wrong)
			else:
				node = Node(is_leaf=False, criterion=best_attr, threshold=threshold_, children=children)
			return node

	def get_main_class(self, S):
		'''
			There is no attribute left. So vote the main label for current data set.
			Input: 
				S: current data set.
			
		'''
		labels = [row[-1] for row in S]
		classes, count = np.unique(labels, return_counts=True)
		max_idx = np.argmax(count)
		pure_degree = round(count[max_idx]/sum(count)*100, ndigits=2)
		return classes[max_idx], pure_degree, np.max(count)

	def __allSameClass(self, data):
		'''
			Check if all rows is the same class
			Input:
				data: current data
			Return:
				False: different classes
				Class_name: if all rows are the same class
		'''
		for row in data:
			if row[-1] != data[0][-1]:
				return False, None
		return True, data[0][-1]

	def __split_data(self, data, attributes):
		'''
			use decision tree algorithm to create tree.
			Input:
				curData: remain data set.
				curAtt
		'''
		#initialize		
		maxEnt = -1*float("inf")
		best_attribute = attributes[0]
		indexOfAttribute = self.attributes.index(best_attribute)
		best_threshold = data[0][indexOfAttribute]
		splitted = [data]

		for attribute in attributes:
			indexOfAttribute = self.attributes.index(attribute)
	
			data.sort(key = lambda x: x[indexOfAttribute])
			for j in range(0, len(data) - 1):
				if data[j][indexOfAttribute] != data[j+1][indexOfAttribute]:
					threshold = data[j][indexOfAttribute]
					less = []
					greater = []
					for row in data:
						if(row[indexOfAttribute] > threshold):
							greater.append(row)
						else:
							less.append(row)
					e = self.gain(data, [less, greater])
					if e >= maxEnt:
						splitted = [less, greater]
						maxEnt = e
						best_attribute = attribute
						best_threshold = threshold
		return (best_attribute,best_threshold,splitted)

	def gain(self,S, Sis):
		E_S = self.entropy(S)
		
		if len(Sis[0])==0 or len(Sis[1])==0:
			return E_S-0

		total_E_Si = sum([len(Si)/len(S)*self.entropy(Si)  for Si in Sis])

		Gain = E_S - total_E_Si
		return Gain

	def entropy(self, S):
		labels = [row[-1] for row in S]
		S = len(labels)

		_,Si = np.unique(labels, return_counts=True)
		Si = list(Si)
		entropy = -sum([si_/S * math.log(si_/S,2) for si_ in Si])
		
		return entropy

	def fit(self, data):
		'''
		this function is to predict with given data.
		
		Input:
			data: one sample, example: [10,20,16,5,51]
		Return: 
			label of this sample. Example: "Yes"
		'''
		node: Node = self.tree
		considered_attr = 0
		while (considered_attr < self.numAttributes):
			if len(node.children) == 0:
				return node.label
			considered_attr += 1
			idx_attr = self.attributes.index(node.criterion)
			if (data[idx_attr] <= node.threshold):
				node = node.children[0]
			else:
				node = node.children[1]
		return node.label


In [33]:
import pandas as pd
import random

class Loader:
    '''
    This class is to load data and feed data into model (C4.5)
    '''
    def __init__(self, path_to_data:str, attributes: list, class_col:str, val_ratio:int=0.1):
        '''
            Input:
                path_to_data: directory containing data
                attributes: list of strings that column name. Example ["HK01", "HK02"]
                class_col: String that is Name of column contain label.
                val_ratio: val / data_total, this will divide data_total into training and validation set.

        '''
        self.filePathToData = path_to_data
        self.val_ratio = val_ratio
        self.attributes = attributes
        self.class_col = class_col
        self.classes=None
        self.num_attributes = len(attributes)
        self.data_total=None
        self.vals = []
        self.datas = []

        self.__load_data()
        self.__split_dataset()
    
    def __load_data(self):
        df = pd.read_excel(self.filePathToData, sheet_name=0, index_col=None, header=0, usecols=self.attributes+[self.class_col])
        self.data_total = df.values.tolist()
        self.classes = list(set(df.iloc[:, -1]))

    def __split_dataset(self):
        # random.shuffle(self.data_total)
        val_ = int(self.val_ratio*10)
        data_ = 10 - val_
        part_ = len(self.data_total)//(val_+data_)
        data_pad = self.data_total+self.data_total
        for part in range(val_+data_ -1):
            self.vals.append(data_pad[part*part_:(part+val_)*part_])
            self.datas.append(data_pad[(part+val_)*part_:(part+val_+data_)*part_])

In [34]:
class Trainer:
    '''
    This class is to make training data with different methods.
    main function of this class: train()
    '''
    def __init__ (self, model, data_loader):
        self.model=model
        self.data_loader = data_loader

    def train(self, is_k_fold=False, prune=False):
        '''
            Input:
                is_k_fold: if True, data will devide into k  parts, fold these parts during training, then compute average acc.
                prune: if True, we will prune last leaf, that means we will merge two branches if they are the same label.
        '''
        model = self.model
        if not is_k_fold:
            model.load_data(attributes=self.data_loader.attributes,
                                 data=self.data_loader.datas[0], classes=self.data_loader.classes)
            model.generate_tree(prune)
            acc = self.__validate(model, self.data_loader.vals[0])
            return model, acc
        else:
            best_k_fold_acc=-1
            best_model=None
            best_acc=None
            best_attrs=[]

            for num_attr in range(1,len(self.data_loader.attributes)+1):
                best_model_, best_acc_, k_fold_acc_ = self.__k_fold_validate(
                    model, self.data_loader.attributes[:num_attr], prune)
                if best_k_fold_acc < k_fold_acc_:
                    best_k_fold_acc = k_fold_acc_
                    best_model = best_model_
                    best_acc = best_acc_
                    best_attrs = self.data_loader.attributes[:num_attr]
            return best_k_fold_acc, best_model, best_acc, best_attrs

    def __validate(self, model, val):
        '''
        compute accuracy of model. 
        
        Return:
            number of true prediction / total prediction
        '''
        hit=0
        for example in val:
            data = example[:-1]
            label = example[-1]
            predict = model.fit(data)
            if (predict == label):
                hit+=1
        return hit/len(val)
    
    def __k_fold_validate(self, model, attrs, prune=False):
        scores = []
        best_model = None
        best_acc = -1
        for data, val in zip(self.data_loader.datas, self.data_loader.vals):
            model_ = model
            model_.load_data(attributes=attrs,
                            data=data, classes=self.data_loader.classes)
            model_.generate_tree(prune)
            acc = self.__validate(model_, val)
            scores.append(acc)
            if acc > best_acc:
                best_acc = acc
                best_model = model_
        k_fold_acc = sum(scores)/len(scores)
        return best_model, best_acc, k_fold_acc


In [35]:

loader = Loader(path_to_data="All.xlsx", 
                attributes=["HK01", "HK02", "HK03", "HK04","first4semesters"], 
                class_col="Graduation", 
                val_ratio=0.2)

#k fold
c1 = C45()
trainer1 = Trainer(c1, loader)
best_k_fold_acc, best_model, best_acc, best_attrs = trainer1.train(is_k_fold=True,prune=True)

best_model.printTree()
best_model.draw_tree("tree k-fold")

#Non k fold
c2 = C45()
trainer2 = Trainer(c2, loader)
c2, acc = trainer2.train(is_k_fold=False, prune=True)

c2.printTree()
c2.draw_tree("tree non k-fold")

#predict
input = [22, 10, 20, 10, 62]
predict1 = c1.fit(input)
predict2 = c2.fit(input)

print("Input 1: {} preddict 1: {}".format(input, predict1))
print("Input 2: {} preddict 2: {}".format(input, predict2))



|____first4semesters <= 53
|	|____HK04 <= 10 : No (100.0)
|	|____HK04 > 10
|		|____HK01 <= 15 : No (100.0)
|		|____HK01 > 15
|			|____HK03 <= 9
|			|	|____HK02 <= 4 : No (100.0)
|			|	|____HK02 > 4 : Yes (100.0)
|			|	
|			|____HK03 > 9 : Yes (100.0)
|			
|____first4semesters > 53
	|____HK01 <= 16
	|	|____HK03 <= 25
	|	|	|____HK02 <= 14 : No (0.875)
	|	|	|____HK02 > 14
	|	|		|____HK04 <= 18 : Yes (63.64)
	|	|		|____HK04 > 18 : No (100.0)
	|	|		
	|	|____HK03 > 25
	|		|____HK02 <= 11 : No (50.0)
	|		|____HK02 > 11 : Yes (100.0)
	|		
	|____HK01 > 16
		|____HK02 <= 12
		|	|____HK04 <= 17 : No (0.6666666666666666)
		|	|____HK04 > 17 : Yes (100.0)
		|	
		|____HK02 > 12 : Yes (0.84375)
		
|____first4semesters <= 53
|	|____HK04 <= 10 : No (100.0)
|	|____HK04 > 10
|		|____HK01 <= 16 : No (100.0)
|		|____HK01 > 16
|			|____HK03 <= 9
|			|	|____HK02 <= 4 : No (100.0)
|			|	|____HK02 > 4 : Yes (100.0)
|			|	
|			|____HK03 > 9 : Yes (100.0)
|			
|____first4semesters > 53
	|____HK02 <= 13
	|	|____HK